In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
data_path='TestPort.csv'
trade_plan_path='MasterConfigData\ListTradeTran.xlsx'
trade_sheet='Test2'

asset_comm_path='MasterConfigData\AssetInfo.xlsx'
result_path=f'ReportData\ReportTestPort_{trade_sheet}.xlsx'


In [10]:
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()

#list_symbols=['KKP-SM-CAP','ASP-SME']
print(list_symbols)
df.info()
print(df.tail())


['ASP-SME' 'ASP-THEQ' 'KKP-ACT-EQ-A' 'KKP-SM-CAP' 'TISCOMS-A' 'TISCOFLEXP'
 'KFDYNAMIC' 'TSF-A']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5826 entries, 0 to 5825
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  5826 non-null   object        
 1   date    5826 non-null   datetime64[ns]
 2   price   5826 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 136.7+ KB
     symbol       date  price
5821  TSF-A 2022-12-30  84.53
5822  TSF-A 2023-01-03  85.13
5823  TSF-A 2023-01-04  83.97
5824  TSF-A 2023-01-05  83.92
5825  TSF-A 2023-01-06  84.42


In [11]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})
dfAssetInfo=pd.read_excel(asset_comm_path,engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})

tradeDateList= [ d for d in list(dfPlan['trade_date']) ]

print(tradeDateList)
print("================================================================================================")

print(dfPlan)
print(dfAssetInfo)


[Timestamp('2022-07-22 00:00:00'), Timestamp('2022-08-05 00:00:00'), Timestamp('2022-09-20 00:00:00'), Timestamp('2022-09-23 00:00:00'), Timestamp('2022-10-17 00:00:00'), Timestamp('2022-10-21 00:00:00'), Timestamp('2022-12-13 00:00:00'), Timestamp('2022-12-15 00:00:00')]
  trade_date  portion_pct action
0 2022-07-22         50.0      b
1 2022-08-05         50.0      b
2 2022-09-20         25.0      s
3 2022-09-23         25.0      s
4 2022-10-17         25.0      b
5 2022-10-21         25.0      b
6 2022-12-13         50.0      s
7 2022-12-15         50.0      s
              buy_comm_pct  sell_comm_pct
symbol                                   
ASP-SME               1.25            0.0
ASP-THEQ              1.00            0.0
KKP-SM-CAP            1.00            0.0
TISCOFLEXP            1.00            0.0
TSF-A                 1.05            0.0
TISCOMS-A             1.00            0.0
KFDYNAMIC             0.50            0.5
KKP-ACT-EQ-A          0.50            0.0


In [12]:

# veridf  there are both buy and sell if no sell
#we have to add last reord 100 % as of last dat in Amibroker database
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()
all_portion=0
if buy_sum_act!=sell_sum_act:
    raise Exception("No.Action Buy and Sell  must be the same.")
else:
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    


# day is ordering ? in plan

# verify sum buy =sum sell

# veriry all list in data price are in commision


portion = 150.0 on


In [13]:
TotalBudget=150000
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception("Budet % Portion is not xxxx")
else:
  print(rrr)

0.0


In [15]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty





In [17]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for symbol in list_symbols:
    print(symbol)
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
    
    dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
    dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
    dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)
    
    dfBS=dfBS.drop(columns=['trade_date'])
    #print(dfBS)
    
    print("================Buy=================")
    buyDF=dfBS.query("action=='b'")    
    # cal qury and actual value and  total_val and totl qury
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
    buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
    buyDF['action_value']=buyDF['value']
    buyDF['action_qty']=buyDF['qty']
    print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),2)
    AvgPrice=round(TotalValue/TotalQty,2)
    # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    
    print("=================Sell=================")
    sellDF=dfBS.query("action=='s'")  
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
    sellDF['action_value']=sellDF['value']*-1
    sellDF['action_qty']=sellDF['qty']*-1
    print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),2)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,2)
    
    # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                  'PL':PL,'Percent_PL':PCT_PL
                 }
    dfSummary=dfSummary.append(summary_data,ignore_index=True)
    
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran=dfTran.sort_values(by=['date'])
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)
    
    dict_dfTran[symbol]=dfTran
    


ASP-SME
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-07-22          16.64  3004.807692  50000.0         50.0
1 2022-08-05          17.03  2935.995302  50000.0         50.0
4 2022-10-17          17.43  1434.308663  25000.0         25.0
5 2022-10-21          17.70  1412.429379  25000.0         25.0
=================Sell=================
        date  price_on_comm          qty         value  portion_pct
2 2022-09-20          17.85  1464.590173  26142.934584         25.0
3 2022-09-23          17.92  1464.590173  26245.455896         25.0
6 2022-12-13          17.52  2929.180345  51319.239653         50.0
7 2022-12-15          17.52  2929.180345  51319.239653         50.0
ASP-THEQ
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-07-22          14.35  3484.320557  50000.0         50.0
1 2022-08-05          14.82  3373.819163  50000.0         50.0
4 2022-10-17          14.5

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_9584\728527262.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_9584\728527262.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_9584\728527262.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [18]:
for symbol,tranDF in dict_dfTran.items():
  tranDF['net_value']=tranDF['action_value'].cumsum(axis = 0) 
  tranDF['net_qty']=tranDF['action_qty'].cumsum(axis = 0)  
  tranDF['avg_price']=round( tranDF['net_value']/tranDF['net_qty'])
  tranDF=tranDF.sort_values(by=['date'])
  print(tranDF[['date' ,'action','price_on_comm','qty','net_qty','value','net_value','avg_price']])

        date action  price_on_comm          qty       net_qty    value  \
0 2022-07-22      b          16.64  3004.807692  3.004808e+03  50000.0   
1 2022-08-05      b          17.03  2935.995302  5.940803e+03  50000.0   
2 2022-09-20      s          17.85  1464.590173  4.476213e+03  26143.0   
3 2022-09-23      s          17.92  1464.590173  3.011623e+03  26245.0   
4 2022-10-17      b          17.43  1434.308663  4.445931e+03  25000.0   
5 2022-10-21      b          17.70  1412.429379  5.858361e+03  25000.0   
6 2022-12-13      s          17.52  2929.180345  2.929180e+03  51319.0   
7 2022-12-15      s          17.52  2929.180345  9.094947e-13  51319.0   

   net_value     avg_price  
0    50000.0  1.700000e+01  
1   100000.0  1.700000e+01  
2    73857.0  1.600000e+01  
3    47612.0  1.600000e+01  
4    72612.0  1.600000e+01  
5    97612.0  1.700000e+01  
6    46293.0  1.600000e+01  
7    -5026.0 -5.526145e+15  
        date action  price_on_comm          qty       net_qty    value  

In [19]:
writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 
dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)
print(dfSummary)
for symbol,tranDF in dict_dfTran.items():
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
 #print(tranDF)
writer.save()

writer.close()

print("Successfully")


         Symbol  BuyValue  AvgBuyPrice  SellValue  AvgSellPrice      PL  \
7         TSF-A  150000.0        78.24  158703.08         82.78  8703.0   
5    TISCOFLEXP  150000.0        58.16  157862.04         61.21  7862.0   
3    KKP-SM-CAP  150000.0        16.97  155621.39         17.61  5621.0   
6     KFDYNAMIC  150000.0        37.93  155236.45         39.25  5236.0   
0       ASP-SME  150000.0        17.07  155026.87         17.64  5027.0   
4     TISCOMS-A  150000.0        20.01  153651.41         20.50  3651.0   
2  KKP-ACT-EQ-A  150000.0        18.52  153531.97         18.96  3532.0   
1      ASP-THEQ  150000.0        14.62  152633.43         14.88  2633.0   

   Percent_PL  
7        5.80  
5        5.24  
3        3.75  
6        3.49  
0        3.35  
4        2.43  
2        2.35  
1        1.76  


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_9584\1572397904.py:8: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


Successfully


D:\ProgramData\Anaconda3\envs\myquant_exp\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
